# Keras cnn + GloVe + Dropout + Early Stopping = [0.048 LB]

#### 2 Jan 2018
***

## **Summary**

This model, nicknamed the_Detoxinator, uses pretrained Glove 840B with Adam optimizer and a learning rate of 0.0001. The model implements early stopping using a stratified validation set (25% of training set). Training was done on a GPU (Crestle). If I remember correctly, training did not take more than 30 minutes.

Please note that this script is written to run locally, not in the Kaggle kernel environment. 


#### **The CNN architecture is as follows:**

model = Sequential()<br>
e = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=500,trainable=False)<br>
model.add(e)<br>
model.add(Conv1D(128, 3, activation='relu'))<br>
model.add(MaxPooling1D(3))<br>
model.add(Dropout(0.2))<br>
model.add(Conv1D(64, 3, activation='relu'))<br>
model.add(MaxPooling1D(3))<br>
model.add(Dropout(0.2))<br>
model.add(Conv1D(64, 3, activation='relu'))<br>
model.add(Dropout(0.2))<br>
model.add(Flatten())<br>
model.add(Dense(1, activation='sigmoid'))<br>

![](http://)The public LB score was 0.048. I found that this score could be improved to 0.046 by averaging the predictions of this cnn model with the predictions generated by another logistic regression model that I created. The following kernel explains how to easily do this:

https://www.kaggle.com/jhoward/minimal-lstm-nb-svm-baseline-ensemble-lb-0-044

Using a weighted average (0.6 x cnn_model + 0.4 x logistic_regression_model) further improved the public LB score to 0.045.
***

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.layers import BatchNormalization, Flatten, Conv1D, MaxPooling1D
from keras.layers import Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# read in the data

#df_train = pd.read_csv('train.csv.zip')
#df_test = pd.read_csv('test.csv.zip')

df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')

print(df_train.shape)
print(df_test.shape)

In [ ]:
# combine the train and test sets for encoding and padding

train_len = len(df_train)
df_combined =  pd.concat(objs=[df_train, df_test], axis=0).reset_index(drop=True)

print(df_combined.shape)

In [ ]:
# define text data
docs_combined = df_combined['comment_text'].astype(str)

# initialize the tokenizer
t = Tokenizer()
t.fit_on_texts(docs_combined)
vocab_size = len(t.word_index) + 1

# integer encode the text data
encoded_docs = t.texts_to_sequences(docs_combined)

# pad the vectors to create uniform length
padded_docs_combined = pad_sequences(encoded_docs, maxlen=500, padding='post')

In [ ]:
# seperate the train and test sets

df_train_padded = padded_docs_combined[:train_len]
df_test_padded = padded_docs_combined[train_len:]

print(df_train_padded.shape)
print(df_test_padded.shape)

### **Load the GloVe embeddings**

The code for processing the Glove embeddings is taken from this excellent blog post:

https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

I made some minor changes to suit glove840B and to fix a "could not convert string to float" error.

In [ ]:
# load the glove840B embedding into memory after downloading and unzippping

embeddings_index = dict()
f = open('glove.840B.300d.txt')

for line in f:
    # Note: use split(' ') instead of split() if you get an error.
	values = line.split(' ')
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

### **Define X and y**

In [ ]:
X = df_train_padded
X_test = df_test_padded

# target columns
y_toxic = df_train['toxic']
y_severe_toxic = df_train['severe_toxic']
y_obscene = df_train['obscene']
y_threat = df_train['threat']
y_insult = df_train['insult']
y_identity_hate = df_train['identity_hate']

### **Train and generate predictions for each of the 6 target columns:**

In [ ]:
# create a list of the target columns
target_cols = [y_toxic,y_severe_toxic,y_obscene,y_threat,y_insult,y_identity_hate]

preds = []

for col in target_cols:
    
    print('\n')
    
    # set the value of y
    y = col
    
    # create a stratified split
    X_train, X_eval, y_train ,y_eval = train_test_split(X, y,test_size=0.25,shuffle=True,
                                                    random_state=5,stratify=y)

    # cnn model
    model = Sequential()
    e = Embedding(vocab_size, 300, weights=[embedding_matrix], 
                  input_length=500, trainable=False)
    model.add(e)
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(3))
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))


    # compile the model
    Adam_opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=Adam_opt, loss='binary_crossentropy', metrics=['acc'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')
    save_best = ModelCheckpoint('toxic.hdf', save_best_only=True, 
                               monitor='val_loss', mode='min')

    history = model.fit(X_train, y_train, validation_data=(X_eval, y_eval),
                        epochs=100, verbose=1,callbacks=[early_stopping,save_best])

    
    # make a prediction on y (target column)
    model.load_weights(filepath = 'toxic.hdf')
    predictions = model.predict(X_test)
    y_preds = predictions[:,0]
    
    # append the prediction to a python list
    preds.append(y_preds)


### **Create a submission file**

In [ ]:
df_results = pd.DataFrame({'id':df_test.id,
                            'toxic':preds[0],
                           'severe_toxic':preds[1],
                           'obscene':preds[2],
                           'threat':preds[3],
                           'insult':preds[4],
                           'identity_hate':preds[5]}).set_index('id')

# Pandas automatically sorts the columns alphabetically by column name.
# Therefore, we need to re-order the columns to match the sample submission file.
df_results = df_results[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

# create a submission csv file
df_results.to_csv('kaggle_submission.csv', 
                  columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate']) 

***
### **Resources**


These are a few cnn and nlp resources I found helpful:

- What are word embeddings?<br>
https://www.youtube.com/watch?v=Eku_pbZ3-Mw


- Blog post with a simple example explaining how to use pre trained embeddings:<br>https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/


- Online cnn course:<br>
https://www.coursera.org/learn/convolutional-neural-networks<br>
This course can be taken for free. 


- Lesson 5 notes from the fast.ai course:<br>
http://wiki.fast.ai/index.php/Lesson_5_Notes


- GloVe: Global Vectors for Word Representation<br>
Jeffrey Pennington, Richard Socher, and Christopher D. Manning. 2014.<br>
https://nlp.stanford.edu/projects/glove/


- Machine learning with text<br>
https://www.youtube.com/watch?v=ZiKMIuYidY0


- NLTK Tutorial series<br>
https://pythonprogramming.net/tokenizing-words-sentences-nltk-tutorial/

***
This competition is a great learning experience. Thank you to all who have been commenting and publishing.

Happy new year!